In [41]:
import pandas as pd
import folium
import numpy as np

In [3]:
bars_df = pd.read_csv('bar_csvs/bar_list_2024.csv', index_col=False)

In [4]:
bars_df = bars_df.rename(columns={"Unnamed: 0": "Bar ID"})

In [5]:
bars_df['Priority'] = 0

In [6]:
updates = {
    51: 1,
    3: 2,
    27: 3,
    143: 4,
    155: 5,
    158: 6,
    95: 7,
    193: 8,
    107: 9,
    42: 10,
    127: 11,
    192: 12,
    196: 13,
    187: 14,
    118: 15,
    72: 16,
    162: 17,
    63: 18,
    55: 19,
    141: 20,
    28: 21,
    77: 22,
    57: 23,
    23: 24,
    157: 25,
    1: 26,
    197: 27,
    24: 28,
    83: 29, 
    122: 30,
    52: 31
}

for row, priority in updates.items():
    bars_df.loc[row, 'Priority'] = priority

In [7]:
drinks_df = pd.read_csv('drinks_df_preferences.csv')

In [8]:
drinks_df = drinks_df.drop(columns=['Unnamed: 0'])

In [9]:
drinks_df['Total'] = drinks_df['Preferences'] + drinks_df['Temp Preferences']

In [10]:
map_center = [bars_df['Latitude'].mean(), bars_df['Longitude'].mean()]
map = folium.Map(location=map_center, zoom_start=12)

In [11]:
for idx, row in bars_df.iterrows():
    priority = row['Priority']
    bar_id = row['Bar ID']
    bar = row['Bar Name']

    drink_rows = drinks_df[drinks_df['Bar ID'] == bar_id]

    value_1 = drink_rows['Preferences'].max()
    value_2 = drink_rows['Temp Preferences'].max()
    total_value = value_1 + value_2

    popup = ''
    
    for drink_idx, drink_row in drink_rows.iterrows():
        drink = drink_row['Drink']
        preference = drink_row['Preferences']
        popup += f" Bar: {bar} Drink: {drink} / Preference: {preference} \n"

    # print(priority)
    if 1 <= priority <= 10:
        color = 'red'
    elif 11 <= priority <= 20:
        color = 'green'
    elif 21 <= priority <= 30:
        color = 'blue'
    else:
        color = 'gray' 

    
    
    if (value_1 >= 4 or value_2 >= 4 or total_value >= 6):
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],  # Coordinates
            popup=popup,
            icon=folium.DivIcon(
                html=f"""
                <div style="
                    font-size: 12px; 
                    color: white; 
                    background-color: {color}; 
                    border-radius: 50%; 
                    width: 35px; 
                    height: 25px;
                    text-align: center;
                    line-height: 25px;">
                    ({int(value_1)},{int(value_2)})
                </div>
                """
            )
        ).add_to(map)

# Ranking

In [13]:
bars_df['Visited'] = False

In [27]:
filtered_df = bars_df[bars_df['Bar Name'].str.contains('Henson', case=False)]

In [28]:
filtered_df

,Bar ID,Bar Name,Address,Phone Number,Description,MON,TUE,WED,THU,FRI,SAT,SUN,Latitude,Longitude,Priority,Visited
51,51,Henson's Bar at Mimi's Hotel,"56-57 Frith Street, London, W1D 3JG",+44 (0) 20 8017 9100,"A sleek, art-lined hotel bar, almost hidden aw...",12PM - 11:30PM,12PM - 11:30PM,12PM - 11:30PM,12PM - 11:30PM,12PM - 12AM,12PM - 12AM,12PM - 10:30PM,51.514023,-0.132093,1,False


In [29]:
# List of Bar IDs where 'Visited' should be True
visited_bar_ids = [28, 143, 55, 63, 148, 107, 51]

# Set 'Visited' to True where 'Bar ID' is in the list
bars_df.loc[bars_df['Bar ID'].isin(visited_bar_ids), 'Visited'] = True

In [31]:
visited_bars_df = bars_df[bars_df['Visited'] == True]

In [42]:
bars_df['Ranking'] = np.nan

In [43]:
updates = {
    51: 1,
    143: 2,
    55: 3,
    28: 4,
    63: 5,
    107: 6,
    148: 7
}

for row, ranking in updates.items():
    bars_df.loc[row, 'Ranking'] = ranking

In [71]:
visited_bars_df = bars_df[bars_df['Visited'] == True]

In [78]:
v_center = [visited_bars_df['Latitude'].mean(), visited_bars_df['Longitude'].mean()]
v = folium.Map(location=v_center, zoom_start=12)

for idx, row in visited_bars_df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']], 
        icon=folium.DivIcon(
            html=f"""
            <div style="
                font-size: 12px; 
                color: white; 
                background-color: #000000; 
                border-radius: 50%; 
                width: 25px; 
                height: 25px;
                text-align: center;
                line-height: 25px;">
                {int(row['Ranking'])}
            </div>
            """
        )
    ).add_to(v)

In [79]:
v